In [21]:
import psycopg2
from config import *

In [22]:
cred = params # load credentials from config file

# connect to the remote postgres database server
connection = psycopg2.connect(**cred)
cursor = connection.cursor()

### Average number of pull requests per contributor who did at least 1 pull request (rounded to the nearest integer).

In [23]:
average_PR_per_active_contributor = """SELECT ROUND(AVG(pull_request_count),0)
                  FROM
                  (SELECT
                      user_id,
                      count(*) AS pull_request_count
                  FROM pulls
                  GROUP BY
                  user_id) average_PR_per_active_contributor;
                """

cursor.execute(average_PR_per_active_contributor)
records = cursor.fetchall()
for record in records:
    print(f"In the pandas repository, an pull request contributor make {record[0]} pull requests on average.")

In the pandas repository, an pull request contributor make 7 pull requests on average.


### Average number of pull requests per total active contributor (rounded to the nearest integer)

In [24]:
average_PR_per_contributor = """
                SELECT ((
                (SELECT COUNT(pulls.user_id)
                    FROM pulls) /
                (SELECT COUNT(contributors.id)
                    FROM contributors) )) AS average_PR_per_contributor
                """
cursor.execute(average_PR_per_contributor)
records = cursor.fetchall()
for record in records:
    print(f"In the pandas repository, the average number of pull requests per total active contributor is {record[0]}.")

In the pandas repository, the average number of pull requests per total active contributor is 57.


### Maximum number of commits per week

In [25]:
max_commit_per_week = """
                    SELECT date(date_trunc('week', commit_date)) AS weekly_commits,
                            COUNT(*)
                    FROM 
                        commits
                    GROUP BY 
                        weekly_commits
                    ORDER BY 
                        count(*) desc LIMIT 1; 
                    """
cursor.execute(max_commit_per_week)
records = cursor.fetchall()
for record in records:
    print(f"For pandas repository, the maximum number of commits in a week is {record[1]}. It was in the week of {record[0]}.")

For pandas repository, the maximum number of commits in a week is 168. It was in the week of 2019-12-30.


### Minimum number of days of an issue being open

In [26]:
open_issues_days = """
                    SELECT 
                        closed_at - created_at AS issue_open_days, 
                        count(*) 
                    FROM 
                        issues
                    GROUP BY 
                        issue_open_days 
                    ORDER by 
                        issue_open_days LIMIT 1;
                    """
cursor.execute(open_issues_days)
records = cursor.fetchall()
for record in records:
    print(f"For pandas repository. The minimum number of days of an issue being open is {record[0]}. There are a total of {record[1]} issues that was closed within the same day.")

For pandas repository. The minimum number of days of an issue being open is 0. There are a total of 4500 issues that was closed within the same day.
